In [42]:
# import required packages
# from fastai2.tabular.all import *
import xgboost as xgb
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
from helpers import *

pd.options.display.max_columns = None

In [43]:
# path to project directory
path = Path('./')

In [44]:
# read in training dataset
train_df = pd.read_csv(path/'data/train_v5.csv', index_col=0, dtype={'season':str})

In [45]:
train_df.tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,assists,bonus,bps,clean_sheets,creativity,goals_conceded,goals_scored,ict_index,influence,own_goals,penalties_missed,penalties_saved,red_cards,saves,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba,relative_market_value_team_season,relative_market_value_opponent_team_season
91396,Yves_Bissouma,2,3,88,Brighton and Hove Albion,Newcastle United,0.452974,0.613904,False,0,0,0,3,1,3.9,0,0,0.3,0.0,0,0,0,1,0,314988,3,0,2.0,21711,52550,30839,0,2020-09-20T13:00:00Z,2021,1.0,NaN,NaN
91397,Zack_Steffen,2,1,0,Manchester City,Wolverhampton Wanderers,2.394374,0.798432,False,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0,0,0,3,1,0.0,0,0,0,0,2020-09-21T19:15:00Z,2021,NaN,NaN,NaN
91398,Zeze Steven_Sessegnon,2,2,0,Fulham,Leeds,0.362034,0.294117,False,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0,0,7068,3,4,0.0,-3500,109,3609,0,2020-09-19T14:00:00Z,2021,0.0,NaN,NaN
91399,Çaglar_Söyüncü,2,2,90,Leicester City,Burnley,1.062043,0.344836,True,1,0,0,17,0,20.9,2,0,8.9,32.4,0,0,0,0,0,861165,2,4,36.0,-16165,41879,58044,0,2020-09-20T18:00:00Z,2021,1.0,NaN,NaN
91400,Ørjan_Nyland,2,1,0,Aston Villa,Sheffield United,0.604579,0.354897,True,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0,0,1059481,0,1,0.0,7351,58445,51094,0,2020-09-21T17:00:00Z,2021,1.0,NaN,NaN


In [46]:
train_df.shape

(91401, 37)

In [47]:
# features for model
season = '2021'
cat_vars = ['season', 'position', 'was_home']
cont_vars = ['gw', 'minutes']
dep_var = ['total_points']

# find the latest gameweek
last_gw = train_df['gw'][train_df['season'] == season].max()

if np.isnan(last_gw): 
    next_gw = 1 
else: 
    next_gw = last_gw + 1
    
next_gw

3

In [52]:
# read in remaining_season.csv and sort
remaining_season_df = pd.read_csv(path/'data/remaining_season.csv', 
                                  index_col=0).sort_values(by=['gw', 'team'])

remaining_season_df['season'] = season
remaining_season_df['total_points'] = 0
remaining_season_df['kickoff_time'] = '9999-99-99'

# append remaining season to end of training set
train_df = train_df.append(remaining_season_df, sort=False).reset_index(drop=True)

In [53]:
# add player lag features
lag_train_df, team_lag_vars = team_lag_features(train_df, ['total_points'], ['all', 1, 2, 3, 4, 5, 10])
lag_train_df, player_lag_vars = player_lag_features(lag_train_df, ['total_points'], 
                                                    ['all', 1, 2, 3, 4, 5, 10])

In [55]:
# we want to set gw and season as ordered categorical variables
# need lists with ordered categories
ordered_gws = list(range(1,39))
ordered_seasons = ['1617', '1718', '1819', '1920', '2021']

# set as categories with correct order 
lag_train_df['gw'] = lag_train_df['gw'].astype('category')
lag_train_df['season'] = lag_train_df['season'].astype('category')

lag_train_df['gw'].cat.set_categories(ordered_gws, ordered=True, inplace=True)
lag_train_df['season'].cat.set_categories(ordered_seasons, ordered=True, inplace=True)

In [57]:
# create dataset with latest lag numbers for all future gameweeks
valid_season = '2021'
valid_gw = next_gw
valid_len = 38 - next_gw + 1

train_predict_df, train_idx, valid_idx = create_lag_train(lag_train_df, 
                                                      cat_vars, cont_vars, 
                                                      player_lag_vars, team_lag_vars, dep_var,
                                                      valid_season, valid_gw, valid_len)

In [59]:
train_predict_df[train_predict_df['player'] == 'Kevin_De Bruyne'].tail(50)

,gw,minutes,minutes_last_1,minutes_last_10,minutes_last_2,minutes_last_3,minutes_last_4,minutes_last_5,minutes_last_all,opponent_team,player,position,season,team,total_points,total_points_pg_last_1,total_points_pg_last_10,total_points_pg_last_2,total_points_pg_last_3,total_points_pg_last_4,total_points_pg_last_5,total_points_pg_last_all,total_points_team_pg_last_1,total_points_team_pg_last_10,total_points_team_pg_last_10_opponent,total_points_team_pg_last_1_opponent,total_points_team_pg_last_2,total_points_team_pg_last_2_opponent,total_points_team_pg_last_3,total_points_team_pg_last_3_opponent,total_points_team_pg_last_4,total_points_team_pg_last_4_opponent,total_points_team_pg_last_5,total_points_team_pg_last_5_opponent,total_points_team_pg_last_all,total_points_team_pg_last_all_opponent,was_home
83051,27,90.0,78.0,836.0,168.0,258.0,348.0,410.0,8976.0,Leicester City,Kevin_De Bruyne,3,1920,Manchester City,3,16.153846,8.397129,8.571429,8.023256,6.465517,7.463415,6.437166,72.0,51.0,40.0,50.0,42.0,42.5,46.666667,48.666667,44.25,43.00,50.8,41.0,54.928571,40.842857,False
84180,29,0.0,90.0,836.0,168.0,258.0,348.0,438.0,9066.0,Manchester United,Kevin_De Bruyne,3,1920,Manchester City,0,3.000000,6.674641,9.107143,6.627907,6.724138,5.753425,6.403044,50.0,48.8,49.1,35.0,61.0,54.0,44.666667,60.666667,47.50,58.50,45.4,50.6,54.893617,45.535211,False
84881,30,69.0,0.0,746.0,90.0,168.0,258.0,348.0,9066.0,Arsenal,Kevin_De Bruyne,3,1920,Manchester City,14,NaN,6.997319,3.000000,9.107143,6.627907,6.724138,6.403044,17.0,45.6,45.2,60.0,33.5,54.5,46.333333,63.000000,37.75,58.75,41.4,54.4,54.626761,45.584507,True
84882,30,29.0,69.0,749.0,69.0,159.0,237.0,327.0,9135.0,Burnley,Kevin_De Bruyne,3,1920,Manchester City,1,18.260870,8.050734,18.260870,9.622642,11.772152,9.082569,6.492611,81.0,51.2,40.5,31.0,49.0,39.5,49.333333,50.333333,55.00,47.25,46.4,47.0,54.811189,37.335664,True
85576,31,90.0,29.0,688.0,98.0,98.0,188.0,266.0,9164.0,Chelsea,Kevin_De Bruyne,3,1920,Manchester City,8,3.103448,7.063953,13.775510,13.775510,8.617021,10.827068,6.481886,93.0,53.8,45.0,45.0,87.0,64.5,63.666667,56.000000,60.25,53.00,62.6,45.6,55.076389,49.305556,False
86221,32,90.0,90.0,688.0,119.0,188.0,188.0,278.0,9254.0,Liverpool,Kevin_De Bruyne,3,1920,Manchester City,14,8.000000,7.848837,6.806723,11.010638,11.010638,8.417266,6.496650,29.0,52.4,58.9,91.0,61.0,69.5,67.666667,61.666667,55.00,51.25,54.0,50.6,54.896552,54.744828,True
86872,33,31.0,90.0,716.0,180.0,209.0,278.0,278.0,9344.0,Southampton,Kevin_De Bruyne,3,1920,Manchester City,1,14.000000,8.170391,11.000000,9.904306,11.978417,11.978417,6.568921,82.0,52.9,40.8,48.0,55.5,33.0,68.000000,48.333333,71.25,43.75,60.4,40.4,55.082192,37.315068,False
87525,34,90.0,31.0,657.0,121.0,211.0,240.0,309.0,9375.0,Newcastle United,Kevin_De Bruyne,3,1920,Manchester City,7,2.903226,8.767123,11.157025,9.810427,9.000000,11.067961,6.556800,21.0,51.3,46.1,41.0,51.5,55.0,44.000000,50.333333,56.25,58.50,61.2,58.4,54.850340,39.055046,True
88178,35,63.0,90.0,657.0,121.0,211.0,301.0,330.0,9465.0,Brighton and Hove Albion,Kevin_De Bruyne,3,1920,Manchester City,3,7.000000,8.767123,5.950413,9.383886,8.970100,8.454545,6.561014,83.0,54.0,36.4,28.0,52.0,45.5,62.000000,37.000000,53.75,38.25,61.6,39.2,55.040541,35.500000,False
88837,36,0.0,63.0,630.0,153.0,184.0,274.0,364.0,9528.0,Bournemouth,Kevin_De Bruyne,3,1920,Manchester City,0,4.285714,9.285714,5.882353,5.380435,8.211679,8.159341,6.545970,94.0,62.2,29.8,50.0,88.5,49.0,66.000000,42.333333,70.00,37.50,61.8,34.6,55.302013,37.261745,True


In [61]:
# take a look at the dataframe
train_predict_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111240 entries, 0 to 111239
Data columns (total 37 columns):
gw                                        111240 non-null category
minutes                                   111240 non-null float64
minutes_last_1                            111240 non-null float64
minutes_last_10                           111240 non-null float64
minutes_last_2                            111240 non-null float64
minutes_last_3                            111240 non-null float64
minutes_last_4                            111240 non-null float64
minutes_last_5                            111240 non-null float64
minutes_last_all                          111240 non-null float64
opponent_team                             111240 non-null object
player                                    111240 non-null object
position                                  111240 non-null int64
season                                    111240 non-null category
team                             

In [63]:
# split out dependent variable
X, y = train_predict_df[cat_vars + cont_vars + player_lag_vars + team_lag_vars].copy(), train_predict_df[dep_var].copy()

In [64]:
# since position is categorical, it should be a string
X['position'] = X['position'].apply(str)

# need to transform season
enc = LabelEncoder()
X['season'] = enc.fit_transform(X['season'])
X_dict = X.to_dict("records")

# Create the DictVectorizer object: dv
dv = DictVectorizer(sparse=False, separator='_')

# Apply dv on df: df_encoded
X_encoded = dv.fit_transform(X_dict)

X_df = pd.DataFrame(X_encoded, columns=dv.feature_names_)

In [65]:
# split out training and prediction sets
X_train = X_df.iloc[train_idx]
y_train = y.iloc[train_idx]
X_test = X_df.iloc[valid_idx]
y_test = y.iloc[valid_idx]

In [67]:
# instatiate and train XGB Regressor
xg_reg = xgb.XGBRegressor(objective="reg:squarederror",
                          gamma=0.42, learning_rate=0.047, max_depth=4, n_estimators=171, subsample=0.6)

xg_reg.fit(X_train, y_train)
preds = xg_reg.predict(X_test)

In [68]:
preds

array([-0.02199572, -0.02881628,  0.01572207, ..., -0.04549783,
       -0.01072395, -0.01072395], dtype=float32)

In [69]:
remaining_season_df['predicted_points'] = preds

In [70]:
remaining_season_df

,gw,team,opponent_team,player,position,price,play_proba,was_home,relative_market_value_team,relative_market_value_opponent_team,season,minutes,total_points,kickoff_time,predicted_points
10225,3,Arsenal,Liverpool,Mesut_Özil,3,6.9,1.00,False,1.489462,2.532511,2021,0.000,0,9999-99-99,-0.021996
10226,3,Arsenal,Liverpool,Sokratis_Papastathopoulos,2,4.9,0.75,False,1.489462,2.532511,2021,0.000,0,9999-99-99,-0.028816
10227,3,Arsenal,Liverpool,David_Luiz Moreira Marinho,2,5.5,1.00,False,1.489462,2.532511,2021,0.000,0,9999-99-99,0.015722
10228,3,Arsenal,Liverpool,Pierre-Emerick_Aubameyang,3,11.9,1.00,False,1.489462,2.532511,2021,90.000,0,9999-99-99,4.894105
10229,3,Arsenal,Liverpool,Cédric_Soares,2,4.9,0.75,False,1.489462,2.532511,2021,0.000,0,9999-99-99,0.008059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9909,38,Wolverhampton Wanderers,Manchester United,Fabio_Silva,4,5.5,1.00,True,0.798432,1.840445,2021,6.000,0,9999-99-99,1.123307
9910,38,Wolverhampton Wanderers,Manchester United,Fernando_Marçal,2,5.0,0.75,True,0.798432,1.840445,2021,36.375,0,9999-99-99,1.461360
9911,38,Wolverhampton Wanderers,Manchester United,Vitor_Ferreira,3,5.0,1.00,True,0.798432,1.840445,2021,0.000,0,9999-99-99,-0.045498
9912,38,Wolverhampton Wanderers,Manchester United,Ki-Jana_Hoever,2,4.5,1.00,True,0.798432,1.840445,2021,0.000,0,9999-99-99,-0.010724


In [71]:
remaining_season_df[remaining_season_df['gw'] == next_gw].sort_values(by='predicted_points', ascending=False).head(50)

,gw,team,opponent_team,player,position,price,play_proba,was_home,relative_market_value_team,relative_market_value_opponent_team,season,minutes,total_points,kickoff_time,predicted_points
9989,3,Manchester United,Brighton and Hove Albion,Anthony_Martial,4,9.0,1.0,False,1.840445,0.452974,2021,90.0,0,9999-99-99,7.823677
9988,3,Manchester United,Brighton and Hove Albion,Bruno Miguel_Borges Fernandes,3,10.5,1.0,False,1.840445,0.452974,2021,90.0,0,9999-99-99,7.642737
136,3,Tottenham Hotspur,Newcastle United,Harry_Kane,4,10.5,1.0,True,1.783233,0.613904,2021,86.5,0,9999-99-99,7.324969
138,3,Tottenham Hotspur,Newcastle United,Heung-Min_Son,3,9.0,1.0,True,1.783233,0.613904,2021,90.0,0,9999-99-99,6.921647
273,3,Manchester City,Leicester City,Raheem_Sterling,3,11.5,1.0,True,2.394374,1.062043,2021,81.0,0,9999-99-99,6.913013
279,3,Manchester City,Leicester City,Gabriel Fernando_de Jesus,4,9.5,1.0,True,2.394374,1.062043,2021,90.0,0,9999-99-99,6.662880
269,3,Manchester City,Leicester City,Kevin_De Bruyne,3,11.6,1.0,True,2.394374,1.062043,2021,90.0,0,9999-99-99,6.612574
302,3,Liverpool,Arsenal,Mohamed_Salah,3,12.1,1.0,True,2.532511,1.489462,2021,90.0,0,9999-99-99,6.223185
281,3,Manchester City,Leicester City,Phil_Foden,3,6.5,1.0,True,2.394374,1.062043,2021,90.0,0,9999-99-99,6.030925
10012,3,Southampton,Burnley,Danny_Ings,4,8.4,1.0,False,0.482558,0.344836,2021,90.0,0,9999-99-99,5.463223


In [73]:
# rename previous week's predictions file
# generate previous week's filename
last_gw = next_gw - 1
filename = 'predictions_gw' + str(last_gw) + '.csv'
# windows
! move predictions.csv $filename
# linux
# ! mv predictions.csv $filename

        1 file(s) moved.


In [74]:
# write to predictions.csv
## RENAME PREVIOUS WEEK FIRST
predictions = remaining_season_df[['gw', 'player', 'team', 'position', 'price', 'play_proba', 'predicted_points', 'minutes']]
predictions.to_csv('predictions.csv')

In [108]:
validation_gw_idx(lag_train_df, valid_season, valid_gw, valid_len)

(91401, 111239)

In [51]:
lag_train_df[(lag_train_df['gw'] == valid_gw) & (lag_train_df['season'] == valid_season)].index.min()

nan

In [55]:
lag_train_df.tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,assists,bonus,bps,clean_sheets,creativity,goals_conceded,goals_scored,ict_index,influence,own_goals,penalties_missed,penalties_saved,red_cards,saves,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba,relative_market_value_team_season,relative_market_value_opponent_team_season,total_points_team,total_points_team_last_all,total_points_team_pg_last_all,total_points_team_last_1,total_points_team_pg_last_1,total_points_team_last_2,total_points_team_pg_last_2,total_points_team_last_3,total_points_team_pg_last_3,total_points_team_last_4,total_points_team_pg_last_4,total_points_team_last_5,total_points_team_pg_last_5,total_points_team_last_10,total_points_team_pg_last_10,total_points_team_opponent,total_points_team_last_all_opponent,total_points_team_pg_last_all_opponent,total_points_team_last_1_opponent,total_points_team_pg_last_1_opponent,total_points_team_last_2_opponent,total_points_team_pg_last_2_opponent,total_points_team_last_3_opponent,total_points_team_pg_last_3_opponent,total_points_team_last_4_opponent,total_points_team_pg_last_4_opponent,total_points_team_last_5_opponent,total_points_team_pg_last_5_opponent,total_points_team_last_10_opponent,total_points_team_pg_last_10_opponent,minutes_last_all,minutes_last_1,minutes_last_2,minutes_last_3,minutes_last_4,minutes_last_5,minutes_last_10,total_points_last_all,total_points_pg_last_all,total_points_last_1,total_points_pg_last_1,total_points_last_2,total_points_pg_last_2,total_points_last_3,total_points_pg_last_3,total_points_last_4,total_points_pg_last_4,total_points_last_5,total_points_pg_last_5,total_points_last_10,total_points_pg_last_10,price
150913,Fabio_Silva,38,4,6.000,Wolverhampton Wanderers,Manchester United,0.798432,1.840445,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.5
150914,Fernando_Marçal,38,2,36.375,Wolverhampton Wanderers,Manchester United,0.798432,1.840445,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
150915,Vitor_Ferreira,38,3,0.000,Wolverhampton Wanderers,Manchester United,0.798432,1.840445,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
150916,Ki-Jana_Hoever,38,2,0.000,Wolverhampton Wanderers,Manchester United,0.798432,1.840445,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5
150917,Nélson_Cabral Semedo,38,2,0.000,Wolverhampton Wanderers,Manchester United,0.798432,1.840445,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.5


In [6]:
# parameters for model
season = '2021'
cat_vars = ['gw', 'season', 'position', 'team', 'opponent_team', 'was_home']
cont_vars = ['minutes', 'relative_market_value_team_season', 'relative_market_value_opponent_team_season']
dep_var = ['total_points']

In [7]:
# we want to set gw and season as ordered categorical variables
# need lists with ordered categories
ordered_gws = list(range(1,39))
ordered_seasons = ['1617', '1718', '1819', '1920', '2021']

In [8]:
# set as categories with correct order 
train_df['gw'] = train_df['gw'].astype('category')
train_df['season'] = train_df['season'].astype('category')

train_df['gw'].cat.set_categories(ordered_gws, ordered=True, inplace=True)
train_df['season'].cat.set_categories(ordered_seasons, ordered=True, inplace=True)

In [9]:
train_df['season']

0        1617
1        1617
2        1617
3        1617
4        1617
         ... 
90857    2021
90858    2021
90859    2021
90860    2021
90861    2021
Name: season, Length: 90862, dtype: category
Categories (5, object): [1617 < 1718 < 1819 < 1920 < 2021]

In [10]:
# dependent variable needs to be float
train_df['total_points'] = train_df['total_points'].astype('float64')

In [11]:
train_df[(train_df['team'] == 'Manchester United') & (train_df['player'] == 'David_de Gea')].head(50)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba,relative_market_value_team_season,relative_market_value_opponent_team_season
132,David_de Gea,1,1,90,Manchester United,Bournemouth,NaN,NaN,False,2.0,...,0.0,0,0,0,0,2016-08-14T12:30:00Z,1617,NaN,1.983179,0.384921
658,David_de Gea,2,1,90,Manchester United,Southampton,NaN,NaN,True,6.0,...,0.0,20807,48379,27572,0,2016-08-19T19:00:00Z,1617,NaN,1.983179,0.796805
1198,David_de Gea,3,1,90,Manchester United,Hull City,NaN,NaN,False,6.0,...,0.0,8827,47432,38605,0,2016-08-27T16:30:00Z,1617,NaN,1.983179,0.494447
1751,David_de Gea,4,1,90,Manchester United,Manchester City,NaN,NaN,True,2.0,...,0.0,-14166,39801,53967,0,2016-09-10T11:30:00Z,1617,NaN,1.983179,2.311012
2333,David_de Gea,5,1,90,Manchester United,Watford,NaN,NaN,False,1.0,...,0.0,-8058,19085,27143,0,2016-09-18T11:00:00Z,1617,NaN,1.983179,0.704200
2918,David_de Gea,6,1,90,Manchester United,Leicester City,NaN,NaN,True,2.0,...,0.0,-75576,13288,88864,0,2016-09-24T11:30:00Z,1617,NaN,1.983179,0.650832
3504,David_de Gea,7,1,90,Manchester United,Stoke City,NaN,NaN,True,3.0,...,0.0,-35363,12586,47949,0,2016-10-02T11:00:00Z,1617,NaN,1.983179,0.718705
4093,David_de Gea,8,1,90,Manchester United,Liverpool,NaN,NaN,False,8.0,...,0.0,-72768,2892,75660,0,2016-10-17T19:00:00Z,1617,NaN,1.983179,1.465860
4686,David_de Gea,9,1,90,Manchester United,Chelsea,NaN,NaN,False,0.0,...,0.0,-20024,8617,28641,0,2016-10-23T15:00:00Z,1617,NaN,1.983179,2.243698
5281,David_de Gea,10,1,90,Manchester United,Burnley,NaN,NaN,True,6.0,...,0.0,-24962,8417,33379,0,2016-10-29T14:00:00Z,1617,NaN,1.983179,0.304218


In [12]:
# find the latest gameweek
last_gw = train_df['gw'][train_df['season'] == season].max()

if np.isnan(last_gw): 
    next_gw = 1 
else: 
    next_gw = last_gw + 1
    
next_gw

2

In [13]:
# read in remaining_season.csv
remaining_season_df = pd.read_csv(path/'data/remaining_season.csv', index_col=0)

In [14]:
remaining_season_df.rename(columns={"relative_market_value_team": "relative_market_value_team_season", 
                                    "relative_market_value_opponent_team": "relative_market_value_opponent_team_season"},
                           inplace=True)

In [15]:
# # set as categories with correct order 
# remaining_season_df['gw'] = train_df['gw'].astype('category')
# remaining_season_df['season'] = train_df['season'].astype('category')

# remaining_season_df['gw'].cat.set_categories(ordered_gws, ordered=True, inplace=True)
# remaining_season_df['season'].cat.set_categories(ordered_seasons, ordered=True, inplace=True)

In [16]:
remaining_season_df[(remaining_season_df['team'] == 'Manchester United') & (remaining_season_df['player'] == 'David_de Gea')]

,gw,team,opponent_team,player,position,price,play_proba,was_home,relative_market_value_team_season,relative_market_value_opponent_team_season,season,minutes
124,2,Manchester United,Crystal Palace,David_de Gea,1,5.5,1.0,True,1.883222,0.476734,2021,90.0
739,4,Manchester United,Tottenham Hotspur,David_de Gea,1,5.5,1.0,True,1.883222,1.694101,2021,90.0
1275,6,Manchester United,Chelsea,David_de Gea,1,5.5,1.0,True,1.883222,2.184688,2021,90.0
1510,7,Manchester United,Arsenal,David_de Gea,1,5.5,1.0,True,1.883222,1.516838,2021,90.0
2040,9,Manchester United,West Bromwich Albion,David_de Gea,1,5.5,1.0,True,1.883222,0.185012,2021,90.0
2802,12,Manchester United,Manchester City,David_de Gea,1,5.5,1.0,True,1.883222,2.441779,2021,90.0
3363,14,Manchester United,Leeds,David_de Gea,1,5.5,1.0,True,1.883222,0.300409,2021,90.0
3893,16,Manchester United,Wolverhampton Wanderers,David_de Gea,1,5.5,1.0,True,1.883222,0.788413,2021,90.0
4162,17,Manchester United,Aston Villa,David_de Gea,1,5.5,1.0,True,1.883222,0.594244,2021,90.0
4900,20,Manchester United,Sheffield United,David_de Gea,1,5.5,1.0,True,1.883222,0.361923,2021,90.0


In [17]:
# processors - categorify categorical variables and normalize continuous variables
# fill missing not used because new teams are almost certainly well below the league average
procs=[Categorify, Normalize]
to_nn = TabularPandas(train_df, procs, cat_vars + ['player'], cont_vars, #+ player_lag_vars + team_lag_vars,
                      #splits=splits, 
                      y_names=dep_var)

In [18]:
dls = to_nn.dataloaders()

In [19]:
dls.show_batch()

,gw,season,position,team,opponent_team,was_home,player,minutes,relative_market_value_team_season,relative_market_value_opponent_team_season,total_points
0,25,1718,3,Newcastle United,Burnley,True,Siem_de Jong,-6.982541e-07,0.547176,0.316798,0.0
1,13,1718,1,Chelsea,Liverpool,False,Willy_Caballero,-6.982541e-07,2.125018,1.619155,0.0
2,16,1819,2,West Ham United,Crystal Palace,True,Issa_Diop,9.000000e+01,0.749352,0.634856,0.0
3,35,1718,3,Stoke City,Burnley,True,Joe_Allen,9.000000e+01,0.581587,0.316798,2.0
4,34,1617,3,Sunderland,Middlesbrough,False,Adnan_Januzaj,1.400000e+01,0.418392,0.452793,1.0
5,11,1617,3,Chelsea,Everton,True,Cesc_Fàbregas,-6.982541e-07,2.243698,1.057509,0.0
6,7,1617,3,Hull City,Chelsea,True,Ryan_Mason,9.000000e+01,0.494447,2.243698,2.0
7,4,1819,3,Newcastle United,Manchester City,False,Matt_Ritchie,-6.982541e-07,0.483921,2.540586,0.0
8,38,1718,2,West Bromwich Albion,Crystal Palace,False,Nathan_Ferguson,-6.982541e-07,0.541354,0.635984,0.0
9,7,1617,4,West Bromwich Albion,Sunderland,False,Hal_Robson-Kanu,9.999996e-01,0.428062,0.418392,1.0


In [20]:
# set range of predictions - minimum to current max
max_y = np.max(train_df['total_points'])
min_y = np.min(train_df['total_points'])
y_range = (-1, max_y)

In [21]:
learn = tabular_learner(dls, y_range=y_range, layers=[1000,500], ps=[0.001,0.01], emb_drop=0.04,
                        n_out=1, loss_func=F.mse_loss, metrics=rmse)

In [22]:
learn.fit_one_cycle(7, 1e-2, wd=0.2)

epoch,train_loss,valid_loss,_rmse,time
0,4.333718,None,None,00:50
1,3.973915,None,None,00:53
2,3.771501,None,None,00:55
3,3.608671,None,None,00:53
4,3.530642,None,None,00:52
5,3.563715,None,None,00:56
6,3.111614,None,None,01:01


/opt/conda/envs/fastai/lib/python3.7/site-packages/fastprogress/fastprogress.py:74: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


In [23]:
# cont_vars = ['minutes', 'relative_market_value_team', 'relative_market_value_opponent_team']

In [54]:
# adjustments
# remaining_season_df.loc[remaining_season_df['player'] == 'Raheem_Sterling', 'minutes'] = 90

In [55]:
to_test = TabularPandas(remaining_season_df, procs, cat_vars + ['player'], cont_vars)#, y_names=dep_var)

In [56]:
test_dl = dls.test_dl(remaining_season_df)

In [57]:
preds = learn.get_preds(dl=test_dl)

In [58]:
preds

(tensor([[ 3.1442e+00],
         [ 2.7156e+00],
         [ 2.6822e+00],
         ...,
         [ 1.7940e+00],
         [-1.2281e-03],
         [ 2.1926e-01]]),
 None)

In [59]:
remaining_season_df['predicted_points'] = preds[0].data.numpy().T[0]

In [60]:
remaining_season_df[remaining_season_df['gw'] == next_gw].sort_values(by='predicted_points', ascending=False).head(50)
# remaining_season_df[remaining_season_df['team'] == 'Sergio_Aguero']

,gw,team,opponent_team,player,position,price,play_proba,was_home,relative_market_value_team_season,relative_market_value_opponent_team_season,season,minutes,predicted_points
135,2,Manchester United,Crystal Palace,Bruno Miguel_Borges Fernandes,3,10.5,1.0,True,1.883222,0.476734,2021,83.600000,7.071314
10140,2,Liverpool,Chelsea,Mohamed_Salah,3,12.0,1.0,False,2.394822,2.184688,2021,73.200000,6.489123
136,2,Manchester United,Crystal Palace,Anthony_Martial,4,9.0,1.0,True,1.883222,0.476734,2021,87.600000,6.333624
10027,2,Tottenham Hotspur,Southampton,Harry_Kane,4,10.5,1.0,False,1.694101,0.495869,2021,90.000000,6.146582
104,2,Leicester City,Burnley,Jamie_Vardy,4,10.0,1.0,True,1.029069,0.351663,2021,90.000000,6.081945
63,2,Everton,West Bromwich Albion,Richarlison_de Andrade,4,8.0,1.0,True,1.152919,0.185012,2021,80.800000,6.066139
181,2,Arsenal,West Ham United,Pierre-Emerick_Aubameyang,3,12.0,1.0,True,1.516838,0.682407,2021,78.400000,5.970190
10029,2,Tottenham Hotspur,Southampton,Heung-Min_Son,3,9.0,1.0,False,1.694101,0.495869,2021,78.400000,5.647571
10162,2,Manchester City,Wolverhampton Wanderers,Raheem_Sterling,3,11.5,1.0,False,2.441779,0.788413,2021,90.000000,5.647542
138,2,Manchester United,Crystal Palace,Dean_Henderson,1,5.5,1.0,True,1.883222,0.476734,2021,90.000000,5.599158


In [62]:
# rename previous week's predictions file
# generate previous week's filename
last_gw = next_gw - 1
filename = 'predictions_gw' + str(last_gw) + '.csv'
! mv predictions.csv $filename

In [63]:
# write to predictions.csv
## RENAME PREVIOUS WEEK FIRST
predictions = remaining_season_df[['gw', 'player', 'team', 'position', 'price', 'play_proba', 'predicted_points']]
predictions.to_csv('predictions.csv')